# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/bitcoin/2_bitcoin_feature_pipeline.ipynb)


## 🗒️ This notebook is divided into the following sections:
1. Parsing Data.
2. Feature Group Insertion.

### <span style="color:#ff5f27;"> 📝 Imports</span>

In [ ]:
!pip install -U hopsworks --quiet

!pip install -U unicorn-binance-rest-api --quiet
!pip install -U python-dotenv --quiet
!pip install -U textblob --quiet
!pip install -U vaderSentiment --quiet
!pip install -U tweepy --quiet
!pip install -U plotly --quiet

In [1]:
# Hosted notebook environments may not have the local features package
import os

def need_download_modules():
    if 'google.colab' in str(get_ipython()):
        return True
    if 'HOPSWORKS_PROJECT_ID' in os.environ:
        return True
    return False

if need_download_modules():
    print("Downloading modules")
    os.system('mkdir -p features')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/bitcoin/features/bitcoin_price.py')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/bitcoin/features/tweets.py')
else:
    print("Local environment")

Local environment


In [8]:
# Uncomment and fill in if you are running on Colab
os.environ['TWITTER_API_KEY'] = "uSTbFVcyHvU0TBF75gYGPZ3L8" # 'ZAIeNMmR0iwEhxyrKWzfB3mwT'
os.environ['TWITTER_API_SECRET'] = "k4MJgLot1ciUCmx4mFiEXjgZF6hH7pBwSw2pB5Ri7R98sib0uC" # 'SGNOmB6zebv6IO2I05hiuFypKwv6DRgGi1s5NLi2KT5y9VgMIo'
os.environ['TWITTER_ACCESS_TOKEN'] = "3407876583-mAeYLekmpIou76oJIK7Bl2abGI6GoM9cxYlThny" # '1535178833435021317-9XZOFOHnNUXYuq08JHC3GW0p1Vanom'
os.environ['TWITTER_ACCESS_TOKEN_SECRET'] = "dMrztRakJTOOLRB8KEN2yXvZefuXk33MKFjgADUSeetYz" #'s7lz5HUTBz8yffYGgDiyHBNbXQuPGqMopmN6fmeiyXZOg'

os.environ['BINANCE_API_KEY'] = 'w3IvBp92KAbrmu1ZjjDiYI4Ev5vTqtRTOkr28tkRBS3iH3z4B0FidCk4JxB6dWjZ'
os.environ['BINANCE_API_SECRET'] = 'f1RC3X5yuWbtzQMBVTl0LwDPTacK1BiLPXcUwwfXTenxh4TcJZ04qXAMik8FYmWp'

In [9]:
import pandas as pd
from features import bitcoin_price, tweets

---
## <span style="color:#ff5f27;"> 🧙🏼‍♂️ Parsing Data</span>

You will parse timeseries Bitcoin data from Binance using your own credentials, so you have to get a free Binance account and [create API-keys](https://www.binance.com/en/support/faq/360002502072).

Also, you should [contact Twitter](https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api) for their API-keys.


#### Don't forget to create an `.env` configuration file inside this directory where all the necessary environment variables will be stored:

`TWITTER_API_KEY = "YOUR_API_KEY"`

`TWITTER_API_SECRET = "YOUR_API_KEY"`

`TWITTER_ACCESS_TOKEN = "YOUR_API_KEY"`

`TWITTER_ACCESS_TOKEN_SECRET = "YOUR_API_KEY"`


`BINANCE_API_KEY = "YOUR_API_KEY"`

`BINANCE_API_SECRET = "YOUR_API_KEY"`

> If you done it after you run this notebook, restart the Python Kernel (because `functions.py` does not have these variables in his namespace).

![](images/api_keys_env_file.png)

### <span style='color:#ff5f27'> 📈 Bitcoin Data

In [4]:
# we should take 56+ days because of feature engineering with window aggregations.
df_bitcoin = bitcoin_price.parse_btc_data(number_of_days_ago=57)
df_bitcoin.head(3)

,date,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,unix
0,2024-02-23 00:00:00,51288.42,51548.54,50521.0,50744.15,30545.79544,1.559144e+09,1487039,14610.80906,7.458210e+08,1708642800000
1,2024-02-24 00:00:00,50744.15,51698.00,50585.0,51568.22,16560.42110,8.477934e+08,855015,8462.15627,4.332396e+08,1708729200000
2,2024-02-25 00:00:00,51568.21,51958.55,51279.8,51728.85,18721.63159,9.669511e+08,923992,9544.17672,4.929525e+08,1708815600000


In [5]:
df_bitcoin_processed = bitcoin_price.process_btc_data(df_bitcoin)
df_bitcoin_processed.tail(3)

,date,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,...,exp_std_14_days,momentum_14_days,rate_of_change_14_days,strength_index_14_days,std_56_days,exp_mean_56_days,exp_std_56_days,momentum_56_days,rate_of_change_56_days,strength_index_56_days
55,2024-04-18,61277.38,64117.09,60803.35,63470.08,43601.60918,2.726741e+09,2142511,20870.20705,1.305027e+09,...,3259.915810,-5017.71,-6.414775,44.593135,5129.861015,66726.875601,3956.587429,0.00,25.078615,0.000000
56,2024-04-19,63470.09,65450.00,59600.01,63818.01,69774.30271,4.419893e+09,2828284,34941.50216,2.214810e+09,...,3110.616630,-4002.61,-7.370515,45.380545,4705.824240,66609.483564,3918.524698,13073.86,23.754533,56.075637
57,2024-04-20,63818.01,64268.58,63090.07,63656.41,9509.29230,6.066525e+08,540253,4884.54200,3.116233e+08,...,2967.551959,-5239.59,-8.223685,45.060239,4281.982584,66490.932081,3883.183688,12088.19,23.057849,55.990013


In [6]:
df_bitcoin_processed.date = df_bitcoin_processed.date.astype(str)

### <span style='color:#ff5f27'> 💭 Tweets Data

In [10]:
df_tweets_parsed = tweets.get_last_tweets()
df_tweets_parsed.head()

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

In [ ]:
tweets_textblob = tweets.textblob_processing(df_tweets_parsed)

In [ ]:
tweets_vader = tweets.vader_processing(df_tweets_parsed)

In [ ]:
tweets_textblob.date = tweets_textblob.date.apply(lambda x: x[:10])
tweets_vader.date = tweets_vader.date.apply(lambda x: x[:10])

In [ ]:
tweets_textblob.head()

---

### <span style="color:#ff5f27;"> 📡 Connecting to the Hopsworks Feature Store </span>

In [12]:
import hopsworks

project = hopsworks.login(
    host="127.0.0.1",
    port="8181",
    project="BinanceFunctionCalling",
    api_key_value="If1pZaYvBO4vFrXP.9JmiCchSUYcyjeRT9rYcbl2KcAUvfNW0Wd81ETdeRdVdgSuB1taPB11DwIIx4vkd"
)

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://127.0.0.1:8181/p/122
Connected. Call `.close()` to terminate connection gracefully.


In [13]:
btc_price_fg = fs.get_or_create_feature_group(
    name='bitcoin_price',
    version=1,
)

tweets_textblob_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_textblob',
    version=1,
)

tweets_vader_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_vader',
    version=1,
)

---

### <span style='color:#ff5f27'> 💫 Filling the gap in tweets

In [ ]:
btc_dates = btc_price_fg.read().date.sort_values().reset_index(drop=True).astype(str)

In [ ]:
stored_tweets_df = tweets_textblob_fg.read()

In [ ]:
stored_dates = stored_tweets_df.date.apply(lambda x: str(x)[:10]).drop_duplicates().sort_values().reset_index(drop=True)

In [ ]:
btc_dates

In [ ]:
stored_dates

In [ ]:
missing_dates = list(set(btc_dates) - set(stored_dates))

In [ ]:
len(missing_dates)

In [ ]:
tweets_textblob_fix = pd.DataFrame(
    {
        "date": missing_dates,
        "subjectivity": [1] * len(missing_dates),
        "polarity": [1] * len(missing_dates),
    })

In [ ]:
tweets_vader_fix = pd.DataFrame(
    {
        "date": missing_dates,
        "compound": [1] * len(missing_dates),
    })

In [ ]:
tweets_vader_fix

In [ ]:
tweets_vader_fix["unix"] = tweets_vader_fix.date.apply(tweets.convert_date_to_unix)
tweets_textblob_fix["unix"] = tweets_textblob_fix.date.apply(tweets.convert_date_to_unix)

In [ ]:
tweets_vader_fix.sort_values("date")

In [ ]:
tweets_vader_batch = pd.concat([tweets_vader_fix, tweets_vader]).sort_values("date").reset_index(drop=True)
tweets_textblob_batch = pd.concat([tweets_textblob_fix, tweets_textblob]).sort_values("date").reset_index(drop=True)

---

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

### <span style='color:#ff5f27'> 📈 Bitcoin Feature Group

In [14]:
btc_price_fg.insert(df_bitcoin_processed)

Uploading Dataframe: 0.00% |          | Rows 0/58 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: bitcoin_price_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://127.0.0.1:8181/p/122/jobs/named/bitcoin_price_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f974860d630>, None)

### <span style='color:#ff5f27'> 💭 Tweets Feature Groups

In [ ]:
tweets_textblob_fg.insert(tweets_textblob_batch)

In [ ]:
tweets_vader_fg.insert(tweets_vader_batch)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline </span>

In the next notebook you will create a feature view, training dataset, train a model and register it in Hopsworks Model Registry.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/bitcoin/3_bitcoin_training_pipeline.ipynb)